In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchinfo import summary

In [2]:
import dagshub
dagshub.init(repo_owner='quangbhdang', repo_name='COSC2984-SMNA-Assignment', mlflow=True)

Accessing as quangbhdang

Initialized MLflow to track repo "quangbhdang/COSC2984-SMNA-Assignment"

Repository quangbhdang/COSC2984-SMNA-Assignment initialized!

In [3]:
# # Upload using the DagsHub client, to a DVC tracked folder also called "data".
# # Follow the instructions that appear to authorize the request.
# from dagshub import upload_files

# dagshub.upload_files('quangbhdang/COSC2984-SMNA-Assignment', 'Dataset/So-Spam/')

Output()

UpdateNotAllowedError: Cannot update existing 'Dataset/So-Spam/relation.csv.gz' file without specifying last_commit

In [3]:
def train(epoch, model):
    model.train()
    output = model(des_tensor,tweets_tensor,num_prop,category_prop,edge_index,edge_type)
    loss_train = loss(output[train_idx], labels[train_idx])
    acc_train = accuracy(output[train_idx], labels[train_idx])
    acc_val = accuracy(output[val_idx], labels[val_idx])
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    print('Epoch: {:04d}'.format(epoch+1),
        'loss_train: {:.4f}'.format(loss_train.item()),
        'acc_train: {:.4f}'.format(acc_train.item()),
        'acc_val: {:.4f}'.format(acc_val.item()),)
    return acc_train,loss_train

def test(model):
    model.eval()
    output = model(des_tensor,tweets_tensor,num_prop,category_prop,edge_index,edge_type)
    loss_test = loss(output[test_idx], labels[test_idx])
    acc_test = accuracy(output[test_idx], labels[test_idx])
    output=output.max(1)[1].to('cpu').detach().numpy()
    label=labels.to('cpu').detach().numpy()
    f1=f1_score(label[test_idx],output[test_idx])
    mcc=matthews_corrcoef(label[test_idx], output[test_idx])
    print("Test set results:",
            "test_loss= {:.4f}".format(loss_test.item()),
            "test_accuracy= {:.4f}".format(acc_test.item()),
            "f1_score= {:.4f}".format(f1),
            "mcc= {:.4f}".format(mcc),
            )
    return acc_test,loss_test,f1, mcc

In [4]:
from Dataset.Dataset import Twibot20
from model import BotRGCN
import torch
from torch import nn
from utils import accuracy, init_weights

from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
embedding_size,dropout,lr,weight_decay=128,0.3,1e-3,5e-3

dataset= Twibot20(device=device ,process=False)
des_tensor,tweets_tensor,num_prop,category_prop,edge_index,edge_type,labels,train_idx,val_idx,test_idx=dataset.dataloader()

botRGCN=BotRGCN(num_prop_size=5,cat_prop_size=3,embedding_dimension=embedding_size).to(device)

loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(botRGCN.parameters(),
                    lr=lr,weight_decay=weight_decay)

botRGCN.apply(init_weights)

epochs=100

for epoch in range(epochs):
    acc_train, loss_train = train(epoch,botRGCN)
    
acc_test, loss_test, f1, mcc = test(botRGCN)

Loading labels...   Finished
Running feature1 embedding
Finished
Running feature2 embedding
Finished
Processing feature3...   Finished
Processing feature4...   Finished
Building graph   Finished
Epoch: 0001 loss_train: 1.4881 acc_train: 0.4912 acc_val: 0.4841
Epoch: 0002 loss_train: 1.0526 acc_train: 0.5690 acc_val: 0.5810
Epoch: 0003 loss_train: 0.9683 acc_train: 0.6183 acc_val: 0.6334
Epoch: 0004 loss_train: 0.8082 acc_train: 0.6505 acc_val: 0.6609
Epoch: 0005 loss_train: 0.7274 acc_train: 0.6951 acc_val: 0.6994
Epoch: 0006 loss_train: 0.6946 acc_train: 0.7060 acc_val: 0.7082
Epoch: 0007 loss_train: 0.6533 acc_train: 0.7069 acc_val: 0.6964
Epoch: 0008 loss_train: 0.6515 acc_train: 0.6982 acc_val: 0.6947
Epoch: 0009 loss_train: 0.6026 acc_train: 0.7072 acc_val: 0.6964
Epoch: 0010 loss_train: 0.5963 acc_train: 0.7173 acc_val: 0.7070
Epoch: 0011 loss_train: 0.5613 acc_train: 0.7311 acc_val: 0.7268
Epoch: 0012 loss_train: 0.5503 acc_train: 0.7589 acc_val: 0.7400
Epoch: 0013 loss_train: 0

In [5]:
import mlflow
import mlflow.pytorch
import uuid

# Optional: set a custom or unique experiment name
experiment_name = f"Dang_TwiBot20_{uuid.uuid4().hex[:6]}"
mlflow.set_experiment(experiment_name)    

2025/05/22 10:37:26 INFO mlflow.tracking.fluent: Experiment with name 'Dang_TwiBot20_6fd58f' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/b365244b730e4023ae6d4ac7cfaba73a', creation_time=1747885046809, experiment_id='6', last_update_time=1747885046809, lifecycle_stage='active', name='Dang_TwiBot20_6fd58f', tags={}>

In [6]:
# Logging experiment results
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_params({
        "embedding_size": embedding_size,
        "dropout": dropout,
        "learning_rate": lr,
        "weight_decay": weight_decay,
        "epochs": epochs
    })

    # Log final test metrics (replace with your actual variable values)
    mlflow.log_metrics({
        "train_accuracy":acc_train.item() if isinstance(acc_train, torch.Tensor) else acc_train,
        "train_loss": loss_train.item() if isinstance(acc_train, torch.Tensor) else loss_train,
        "test_accuracy": acc_test.item() if isinstance(acc_test, torch.Tensor) else acc_test,
        "test_loss": loss_test.item() if isinstance(loss_test, torch.Tensor) else loss_test,
        "f1_score": f1,
        "matthew_coeff": mcc
        
    })

    # Log model summary.
    with open("model_summary.txt", "w") as f:
        f.write(str(summary(botRGCN)))
    mlflow.log_artifact("model_summary.txt")

    # Optionally log model
    mlflow.pytorch.log_model(botRGCN, "BotRGCN")

2025/05/22 10:37:47 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/22 10:37:53 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/22 10:37:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run gaudy-hound-61 at: https://dagshub.com/quangbhdang/COSC2984-SMNA-Assignment.mlflow/#/experiments/6/runs/527242af7ed34ca381e6cbb5bdfb1777
🧪 View experiment at: https://dagshub.com/quangbhdang/COSC2984-SMNA-Assignment.mlflow/#/experiments/6


In [7]:
from model import BotGCN

embedding_size,dropout,lr,weight_decay=128,0.3,1e-3,5e-3

botGCN=BotGCN(num_prop_size=5,cat_prop_size=3,embedding_dimension=embedding_size).to(device)

loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(botGCN.parameters(),
                    lr=lr,weight_decay=weight_decay)

botRGCN.apply(init_weights)

epochs=100

for epoch in range(epochs):
    acc_train, loss_train = train(epoch,botGCN)
    
acc_test, loss_test, f1, mcc = test(botGCN)

Epoch: 0001 loss_train: 0.6961 acc_train: 0.4395 acc_val: 0.4402
Epoch: 0002 loss_train: 0.6890 acc_train: 0.5645 acc_val: 0.5518
Epoch: 0003 loss_train: 0.6845 acc_train: 0.5619 acc_val: 0.5522
Epoch: 0004 loss_train: 0.6797 acc_train: 0.5774 acc_val: 0.5611
Epoch: 0005 loss_train: 0.6739 acc_train: 0.6403 acc_val: 0.6199
Epoch: 0006 loss_train: 0.6674 acc_train: 0.6508 acc_val: 0.6520
Epoch: 0007 loss_train: 0.6587 acc_train: 0.6604 acc_val: 0.6554
Epoch: 0008 loss_train: 0.6508 acc_train: 0.6570 acc_val: 0.6495
Epoch: 0009 loss_train: 0.6412 acc_train: 0.6614 acc_val: 0.6579
Epoch: 0010 loss_train: 0.6307 acc_train: 0.6591 acc_val: 0.6677
Epoch: 0011 loss_train: 0.6199 acc_train: 0.6654 acc_val: 0.6655
Epoch: 0012 loss_train: 0.6072 acc_train: 0.6784 acc_val: 0.6702
Epoch: 0013 loss_train: 0.5965 acc_train: 0.6775 acc_val: 0.6655
Epoch: 0014 loss_train: 0.5835 acc_train: 0.6953 acc_val: 0.6841
Epoch: 0015 loss_train: 0.5808 acc_train: 0.6889 acc_val: 0.6782
Epoch: 0016 loss_train: 0

In [8]:
# Logging experiment results
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_params({
        "embedding_size": embedding_size,
        "dropout": dropout,
        "learning_rate": lr,
        "weight_decay": weight_decay,
        "epochs": epochs,
        "optimiser": "AdamW"
    })

    # Log final test metrics (replace with your actual variable values)
    mlflow.log_metrics({
        "train_accuracy":acc_train.item() if isinstance(acc_train, torch.Tensor) else acc_train,
        "train_loss": loss_train.item() if isinstance(acc_train, torch.Tensor) else loss_train,
        "test_accuracy": acc_test.item() if isinstance(acc_test, torch.Tensor) else acc_test,
        "test_loss": loss_test.item() if isinstance(loss_test, torch.Tensor) else loss_test,
        "f1_score": f1,
        "matthew_coeff": mcc
        
    })

    # Log model summary.
    with open("model_summary.txt", "w") as f:
        f.write(str(summary(botGCN)))
    mlflow.log_artifact("model_summary.txt")

    # Optionally log model
    mlflow.pytorch.log_model(botGCN, "model")

2025/05/22 10:38:24 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/22 10:38:27 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/22 10:38:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run bald-zebra-608 at: https://dagshub.com/quangbhdang/COSC2984-SMNA-Assignment.mlflow/#/experiments/6/runs/bc8296122caa446da63459c981b99bb2
🧪 View experiment at: https://dagshub.com/quangbhdang/COSC2984-SMNA-Assignment.mlflow/#/experiments/6


In [9]:
from model import BotGAT

embedding_size,dropout,lr,weight_decay=128,0.3,1e-3,5e-3

botGAT=BotGAT(num_prop_size=5,cat_prop_size=3,embedding_dimension=embedding_size).to(device)

loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(botGAT.parameters(),
                    lr=lr,weight_decay=weight_decay)

botGAT.apply(init_weights)

epochs=100

for epoch in range(epochs):
    acc_train, loss_train = train(epoch,botGAT)
    
acc_test, loss_test, f1, mcc = test(botGAT)


Epoch: 0001 loss_train: 0.8152 acc_train: 0.5291 acc_val: 0.5188
Epoch: 0002 loss_train: 0.6962 acc_train: 0.5927 acc_val: 0.5725
Epoch: 0003 loss_train: 0.6722 acc_train: 0.6065 acc_val: 0.5877
Epoch: 0004 loss_train: 0.6409 acc_train: 0.6274 acc_val: 0.6144
Epoch: 0005 loss_train: 0.6366 acc_train: 0.6593 acc_val: 0.6541
Epoch: 0006 loss_train: 0.6329 acc_train: 0.6651 acc_val: 0.6638
Epoch: 0007 loss_train: 0.6193 acc_train: 0.6667 acc_val: 0.6664
Epoch: 0008 loss_train: 0.6146 acc_train: 0.6552 acc_val: 0.6562
Epoch: 0009 loss_train: 0.6224 acc_train: 0.6468 acc_val: 0.6368
Epoch: 0010 loss_train: 0.6104 acc_train: 0.6608 acc_val: 0.6537
Epoch: 0011 loss_train: 0.6011 acc_train: 0.6755 acc_val: 0.6630
Epoch: 0012 loss_train: 0.6087 acc_train: 0.6767 acc_val: 0.6774
Epoch: 0013 loss_train: 0.5985 acc_train: 0.6851 acc_val: 0.6863
Epoch: 0014 loss_train: 0.5936 acc_train: 0.6818 acc_val: 0.6846
Epoch: 0015 loss_train: 0.5862 acc_train: 0.6816 acc_val: 0.6778
Epoch: 0016 loss_train: 0

In [10]:
# Logging experiment results
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_params({
        "embedding_size": embedding_size,
        "dropout": dropout,
        "learning_rate": lr,
        "weight_decay": weight_decay,
        "epochs": epochs,
        "optimiser": "AdamW"
    })

    # Log final test metrics (replace with your actual variable values)
    mlflow.log_metrics({
        "train_accuracy":acc_train.item() if isinstance(acc_train, torch.Tensor) else acc_train,
        "train_loss": loss_train.item() if isinstance(acc_train, torch.Tensor) else loss_train,
        "test_accuracy": acc_test.item() if isinstance(acc_test, torch.Tensor) else acc_test,
        "test_loss": loss_test.item() if isinstance(loss_test, torch.Tensor) else loss_test,
        "f1_score": f1,
        "matthew_coeff": mcc
        
    })

    # Log model summary.
    with open("model_summary.txt", "w") as f:
        f.write(str(summary(botGAT)))
    mlflow.log_artifact("model_summary.txt")

    # Optionally log model
    mlflow.pytorch.log_model(botGAT, "model")

2025/05/22 10:38:38 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/22 10:38:42 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/22 10:38:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run melodic-fox-832 at: https://dagshub.com/quangbhdang/COSC2984-SMNA-Assignment.mlflow/#/experiments/6/runs/d11213f31c0749249e28a50f24e2f81c
🧪 View experiment at: https://dagshub.com/quangbhdang/COSC2984-SMNA-Assignment.mlflow/#/experiments/6
